#### Vscode has a Dev Containers extension for creating Dockerized development environments

https://marketplace.visualstudio.com/items?itemName=ms-vscode-remote.remote-containers

This guide includes the instructions for creating a Dev Container environment for running the Shiny App using R 3.6.3.

#### Start by pulling or building the oel8/Dockerfile image

In [ ]:
!docker build -t sec_poc/R .
# or
!docker pull crouchcd/oel8-r:3.6.0

#### Start the docker application with docker-compose

In [ ]:
# start the application stack, which includes a PostgreSQL instance, in detached mode.
# NOTE: it has hardcoded your bind-mounted volume to exist at /Users/<your_user>/Desktop/sec_poc_workspace, you can change that
!docker-compose up -d

Now inside of Vscode (making sure that the dev container extension is installed), you should be able to connect to the R App container.

See the command > Dev Containers: Attach to Running Container...

.vscode start with empty workspace. Add folders as needed (e.g., sec_poc, sec_admin). Save "workspace as..."

#### Install these system packages for better developer experience

In [ ]:
# For tracking file changes. If mounting as a Docker volume, the .git/ folder should already be initialized to the sec_poc repo.
!sudo dnf install -y git
# Add GitHub CLI for `gh auth login`
!sudo dnf config-manager --add-repo https://cli.github.com/packages/rpm/gh-cli.repo
!sudo dnf install -y gh --repo gh-cli
# For watching file changes while running shiny::runApp. See sec_poc/watch.sh
!sudo dnf install -y inotify-tools
# For installing/building Python via Pyenv. See .python-version for expected python version.
!sudo dnf install -y ncurses-devel readline-devel libffi-devel
# Installs psql to connect to the Postgres container
# Install the repository RPM:
!sudo dnf install -y https://download.postgresql.org/pub/repos/yum/reporpms/EL-8-x86_64/pgdg-redhat-repo-latest.noarch.rpm
# Disable the built-in PostgreSQL module:
!sudo dnf -qy module disable postgresql
# Install PostgreSQL Client (psql):
!sudo dnf install -y postgresql16-client


#### Missing R packages

In [ ]:
!Rscript -e 'pak::pkg_install("cran/config@0.3.2")'

#### R Debugger Extension

See the recommended extensions in [.vscode/extensions.json](../../.vscode/extensions.json)

#### Additional software

- Pyenv

Python is used for database ETL scripts.

In [ ]:
!curl https://pyenv.run | bash
# Add the init commands to ~/.bashrc

- Starship (Customizable shell prompt) (optional)

In [ ]:
!curl -sS https://starship.rs/install.sh | sh
# Add the init command to ~/.bashrc

#### Docker networking

Assuming that the R App and PostgreSQL containers are in the same network, the container name can be used as the hostname when connecting to the container. Or the hostname can be `host.docker.internal`. `host.docker.internal` can also be used to connect from the host to the container on an exposed port. For example, `psql -h host.docker.internal -d sec -U secapp`

#### PostgreSQL

It's easier to save the connection params in ~/.pgpass so that psql doesn't prompt for password everytime

#### Jupyter

See this guide for using R in Jupyter Notebooks. Make sure that Jupyter is installed in the Python environment.
https://stackoverflow.com/questions/56497422/using-jupyter-r-kernel-with-visual-studio-code